<a href="https://colab.research.google.com/github/Benish-Lezdo/ai_learnings/blob/main/colpoli_document_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColPali Example



```
# This is formatted as code
```



In [ ]:
!pip install --upgrade byaldi
!pip install  poppler-utils
!pip install -q pdf2image git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn




In [ ]:
!pip install --upgrade byaldi
!pip install  poppler-utils
!pip install -q pdf2image git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn
!pip install -q pdf2image

!sudo apt-get install -y poppler-utils

from byaldi import RAGMultiModalModel
from huggingface_hub import notebook_login
from pdf2image import convert_from_path

images = convert_from_path("/content/injury_report.pdf")
images[0]







In [20]:
from byaldi import RAGMultiModalModel


# model = RAGMultiModalModel.from_pretrained("vidore/colpali")
model = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2")

model.index(
    input_path="/content/injury_report.pdf",
    index_name= "image_index",
    store_collection_with_index=False,
    overwrite=True,
)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: index_name must be specified to create a new index.